Analyze the output of metagenome-atlas
======================================

In [ ]:
import os
os.environ['QT_QPA_PLATFORM']='offscreen' # ete3 has some interactive part, but we don't have acces to them here

# supress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load libraries

%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

#load my scripts
from utils.mag_scripts import * 
from utils.barplots import * 

import ete3



In [ ]:
# Define filepaths

atlas_wd_folder = "../Example/"

taxonomy_file = os.path.join(atlas_wd_folder,"genomes/taxonomy/gtdb_taxonomy.tsv")
tree_file = os.path.join(atlas_wd_folder,"genomes/tree/gtdbtk.bac120.nwk")
quality_file= os.path.join(atlas_wd_folder,"genomes/checkm/completeness.tsv")
counts_file= os.path.join(atlas_wd_folder,"genomes/counts/raw_counts_genomes.tsv")
abundance_file = os.path.join(atlas_wd_folder,"genomes/counts/median_coverage_genomes.tsv")
readstats_file= os.path.join(atlas_wd_folder,"stats/read_counts.tsv")
keggmodules_file = os.path.join(atlas_wd_folder,"genomes/annotations/dram/kegg_modules.tsv")


# Taxonomy

In [ ]:
Tax= pd.read_table(taxonomy_file,index_col=0)
Tax.head()

In [ ]:
# create a short label for each species
Labels=Tax.ffill(axis=1).species.copy()
Labels.loc[Tax.species.isnull()]+= ' '+ Labels.index[Tax.species.isnull()]

## Draw tree

In [ ]:
T= ete3.Tree(tree_file)

In [ ]:
unique_phyla= Tax.phylum.unique()
phyla_colors= dict(zip(unique_phyla, 
['#bf423f',
 '#bf973f',
 '#91bf3f',
 '#3fbf42',
 '#3fbf97',
 '#3f91bf',
 '#423fbf',
 '#973fbf',
 '#bf3f91']))
    

def layout(node):
    node.img_style["size"] = 0
    if node.is_leaf():
        L= ete3.TextFace(Labels.loc[node.name])
        ete3.add_face_to_node(L, node, 0, position="branch-right")        
        node.set_style(ete3.NodeStyle(bgcolor= phyla_colors[Tax.loc[node.name,'phylum']]))
        


ts=ete3.TreeStyle()
ts.mode='c'
ts.show_leaf_name=False
ts.scale = 200

for ph in unique_phyla:
    ts.title.add_face(ete3.CircleFace(radius=15,color= phyla_colors[ph] ), column=0)
    ts.title.add_face(ete3.TextFace(ph, fsize=15), column=1)

T.render('%%inline',tree_style=ts,layout=layout)

# Genome quality 

In [ ]:
genome_quality= pd.read_table(quality_file,index_col=0)

genome_quality['Quality_Score']= genome_quality.eval('Completeness -5*Contamination')
genome_quality['Lineage']= genome_quality['Marker lineage'].map(lambda s: s.split()[0])

genome_quality['Id']= genome_quality.index

genome_quality= genome_quality.join(Tax)
genome_quality['Name']= Labels

In [ ]:

xscale = alt.Scale(domain=(0, 10))
yscale = alt.Scale(domain=(50, 100))

alt.Chart(genome_quality).mark_circle(opacity= .6).encode(
    alt.X('Contamination', scale=xscale, title='Contamination [%]'),
    alt.Y('Completeness', scale=yscale, title='Completeness [%]'),
    color='phylum',
    tooltip=['Name', 'Id', 'Contamination','Completeness' ]
).interactive()

# Abundance

## Mapping rate

In [ ]:
# calculate mapping  rate
Counts= pd.read_csv(counts_file,index_col=0,sep='\t').T
read_stats= pd.read_csv(readstats_file,index_col=0,sep='\t').query('Step=="QC"')


mapped_reads = Counts.sum(1)
total_reads = read_stats.eval('Reads_pe *2 + Reads_se')

mapping_rate = mapped_reads/total_reads *100




In [ ]:
f,ax= plt.subplots(figsize=(2,4))
ax.set_ylim([0,100])
ax.set_xlabel('Samples')
sns.swarmplot(y= mapping_rate,ax=ax)

ax.set_title('Mapping rate')


## Relative abundance


For the relative abundance, we take the coverage over the genome, not the raw counts. This implicitly normalizes for genome size. The coverage is calculated as the median of the coverage values calculated in 1kb blocks.

In [ ]:
D = pd.read_table(abundance_file,index_col=0)


#calculate relative abundance

relab = (D.T/D.sum(1)).T

relab.head()

In [ ]:
# get most abundant genomes

counts_per_genome= relab.sum().sort_values()
ax= counts_per_genome[-10:].plot.bar(figsize=(10,5))

_= ax.set_xticklabels(Labels.loc[counts_per_genome.index[-10:]])
ax.set_title('Most abundant genomes')
ax.set_ylabel('Abundance [%]')

### Typical bar chart

In [ ]:

level='family'

grouped_data =relab.groupby(Tax[level],axis=1).sum()

ax= BarPlot(grouped_data)

ax.legend_.set_title(level,{'weight':'bold'})


# Functional annotation


The relative abundance of functional annotations per sample.

The abundance is calculated as the sum of the relative abundance of all bacteria containing a function.


## Kegg modules produced by Dram

In [ ]:
kegg_modules= pd.read_table(keggmodules_file,index_col=[1,2]).drop('Unnamed: 0',axis=1)
module_names =kegg_modules.module_name.droplevel(0).drop_duplicates()
kegg_modules.head()

In [ ]:
# Calculate module step coverage per genome

step_coverage_threshold= 0.8
module_step_coverage_matrix = kegg_modules.step_coverage.unstack(fill_value=0)
module_step_coverage_matrix= module_step_coverage_matrix.loc[:, module_step_coverage_matrix.max()>0]



cgi= sns.clustermap(module_step_coverage_matrix,
metric='cosine',
figsize=(10,6),row_cluster=False,
center= step_coverage_threshold,cmap= 'RdBu_r',vmax=1,vmin=0)

cgi_bin=sns.clustermap(module_step_coverage_matrix> step_coverage_threshold,
                   figsize=(10,6),
                   row_cluster=False,
                       col_linkage= cgi.dendrogram_col.linkage
                  )



module_presence_matrix = (module_step_coverage_matrix > step_coverage_threshold) *1
#drop all 0 modules
module_presence_matrix= module_presence_matrix.loc[:,module_presence_matrix.max()>0]




In [ ]:
#Todo interactive heatmap

In [ ]:
# Module abundance
# Sum of rel_ab for all species where a module is presence is equel to the matrix multiplication

assert relab.shape[1] == module_presence_matrix.shape[0], "Relab and module matrix shoul dhave the same shape"


module_relab = relab @ module_presence_matrix

sns.clustermap(module_relab, figsize=(10,6),
                   )

